In [1]:
import pandas as pd
import time
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [2]:
from numba import njit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean

@njit
def rolling_mean_14(x):
    return rolling_mean(x, window_size=14)
@njit
def rolling_mean_30(x):
    return rolling_mean(x, window_size=30)

In [3]:
def format_df_to_mlforecast(df, date_col, target_col, unique_id='mean'):
    df_ = df.rename({
        date_col: "ds",
        # target_col: 'y',
    }, axis=1)

    df_['ds'] = pd.to_datetime(df_['ds'])

    df_['y'] = df_[target_col].copy()
    # df_.drop(columns=target_col)

    df_['unique_id'] = unique_id
    return df_

In [4]:
selected_sensors_df = pd.read_csv("../data/selected_sensors2_cleaned.csv", index_col=0)

In [5]:
scenarios_sensors = {
    # 0: 1, 4372603
    # "0_12M_train_7M_test": {"train_start": "2017-03-25", "train_end": "2018-03-25", "test_start": "2018-03-26", "test_end": "2018-10-10"},
    '2': {
        "26M_train":  {"train_start": "2017-04-01", "train_end": "2019-06-01"},
        "24M_train":  {"train_start": "2017-04-01", "train_end": "2019-04-01"},
        "22M_train":  {"train_start": "2017-04-01", "train_end": "2019-02-01"},
        "20M_train":  {"train_start": "2017-04-01", "train_end": "2018-12-01"},
        "18M_train":  {"train_start": "2017-04-01", "train_end": "2018-10-01"},
        "12M_train":  {"train_start": "2017-04-01", "train_end": "2018-04-01"},
        "10M_train":  {"train_start": "2017-04-01", "train_end": "2018-01-25"},
        "8M_train":   {"train_start": "2017-04-01", "train_end": "2017-10-25"},
        
        # Non-Heating Periods
        "NH_3M_train":  {"train_start": "2017-04-15", "train_end": "2017-07-15"},
        "NH_4M_train":  {"train_start": "2017-04-15", "train_end": "2017-08-15"},
        "NH_2M_train":  {"train_start": "2017-04-15", "train_end": "2017-06-15"},
        "NH_1M_train":  {"train_start": "2017-04-15", "train_end": "2017-05-15"},
        "NH_15D_train": {"train_start": "2017-04-15", "train_end": "2017-04-30"},
        "NH_feb_2M_train": {"train_start": "2017-02-15", "train_end": "2017-04-15"},
        "NH_feb_1M_train": {"train_start": "2017-02-15", "train_end": "2017-04-15"},
        "NH_mar_2M_train": {"train_start": "2017-03-15", "train_end": "2017-05-15"},
        "NH_mar_1M_train": {"train_start": "2017-03-15", "train_end": "2017-04-15"},

        # Heating Periods
        "H_5M_train":     {"train_start": "2017-06-01", "train_end": "2017-11-01"},
        "H_3M_jul_train": {"train_start": "2017-07-01", "train_end": "2017-10-10"},
        "H_3M_sep_train": {"train_start": "2017-09-01", "train_end": "2017-12-10"},
        "H_3M_nov_train": {"train_start": "2017-11-01", "train_end": "2018-02-10"},
        },
}
scenarios_sensors['5'] = scenarios_sensors['2'].copy()
scenarios_sensors['6'] = scenarios_sensors['2'].copy()

In [6]:
from MLForecastPipeline import *

In [7]:
def split_data(df, scenario, date_col="ds"):
    """Extracts train and test data based on train end date."""
    train_data = df[df[date_col] <= scenario['train_end']]
    test_start = pd.to_datetime(scenario['train_end']) + pd.Timedelta(days=1)
    test_data = df[df[date_col] >= test_start]
    return train_data, test_data

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import (
    AutoARIMA, AutoETS, AutoCES, AutoTheta, CrostonClassic, CrostonSBA,
    AutoTBATS, SeasonalExponentialSmoothingOptimized, HoltWinters, Holt,
    SeasonalWindowAverage, ADIDA, CrostonOptimized, IMAPA, TSB,
    MSTL, MFLES, OptimizedTheta, DynamicOptimizedTheta, GARCH
)
models_dict = {
    "AutoARIMA": AutoARIMA(season_length=365),
    "AutoETS": AutoETS(season_length=365),
    "AutoCES": AutoCES(season_length=365),
    "AutoTheta": AutoTheta(season_length=365),
    "CrostonClassic": CrostonClassic(),
    "CrostonSBA": CrostonSBA(),
    "AutoTBATS": AutoTBATS(season_length=365),
    "SeasonalExponentialSmoothingOptimized": SeasonalExponentialSmoothingOptimized(season_length=365),
    "HoltWinters": HoltWinters(season_length=365),
    "Holt": Holt(),
    "SeasonalWindowAverage1": SeasonalWindowAverage(season_length=365, window_size=30),
    "SeasonalWindowAverage2": SeasonalWindowAverage(season_length=365, window_size=60),
    "SeasonalWindowAverage3": SeasonalWindowAverage(season_length=365, window_size=15),
    "ADIDA": ADIDA(),
    "CrostonOptimized": CrostonOptimized(),
    "IMAPA": IMAPA(),
    # "TSB": TSB(),
    "MSTL_AdditiveB": MSTL(season_length=[7, 30, 365], trend_forecaster=AutoARIMA()),
    "MSTL_Additive": MSTL(season_length=[7, 365], trend_forecaster=AutoARIMA()),
    "MSTL_MultiplicativeB": MSTL(season_length=[7, 30, 365], trend_forecaster=AutoARIMA()),
    "MSTL_Multiplicative": MSTL(season_length=[30, 365], trend_forecaster=AutoARIMA()),
    "MFLES1": MFLES(season_length=365),
    "MFLES2": MFLES(season_length=365, fourier_order=3),
    "OptimizedTheta": OptimizedTheta(season_length=365),
    "DynamicOptimizedTheta1": DynamicOptimizedTheta(season_length=365),
    "DynamicOptimizedTheta2": DynamicOptimizedTheta(season_length=365, decomposition_type='additive'),
    "GARCH": GARCH()
}

class LogTransform:
    def fit_transform(self, y):
        self.min_ = y.min()
        if self.min_ <= 0:
            y = y - self.min_ + 1
        self.transformed_ = np.log(y)
        return self.transformed_

    def transform(self, y):
        if self.min_ <= 0:
            y = y - self.min_ + 1
        return np.log(y)

    def inverse_transform(self, y):
        return np.exp(y) + self.min_ - 1 if self.min_ <= 0 else np.exp(y)

def evaluate_models_statsforecast(train_df, test_df, target_transforms):
    results = []

    log_transform = LogTransform()
    target_transforms.append(log_transform)

    valid_transform_combinations = [()] + list(chain(combinations(target_transforms, 1), combinations(target_transforms, 2)))
    valid_transform_combinations = [tc for tc in valid_transform_combinations if filter_conflicting_transforms(tc)]
    max_test_length = len(test_df)
    test_lengths = [t for t in range(30, 181, 30)] + [240, 300, 360, 480, 600, 720, max_test_length]
    test_lengths = [t for t in test_lengths if t <= max_test_length]
    total_fits = len(valid_transform_combinations) * len(models_dict)
    print(f"Total model fits to run: {total_fits}")
    fit_num = 0

    for model_name, model in models_dict.items():
        for transform_combination in valid_transform_combinations:
            print(f"{fit_num + 1}/{total_fits} Training {model_name} with transforms: {stringify_transform(transform_combination)}...")
            try:
                train_ga = dataframe_to_grouped_array(train_df, 'unique_id', 'y')
                transformed_train_ga = apply_transformations(train_ga, transform_combination)
                transformed_train_df = train_df.copy()
                transformed_train_df['y'] = transformed_train_ga.data
                sf = StatsForecast(models=[model], freq='D', n_jobs=-1)
                sf.fit(transformed_train_df)
                forecasts = sf.predict(h=max_test_length)
                transformed_forecast_ga = dataframe_to_grouped_array(forecasts, 'unique_id', 'y')
                inverse_transformed_forecast_ga = inverse_transformations(transformed_forecast_ga, transform_combination)
                test_df_copy = test_df.copy()
                test_df_copy['forecast'] = inverse_transformed_forecast_ga.data
                error_dict = {}
                for test_length in test_lengths:
                    eval_subset = test_df_copy.iloc[:test_length]
                    error_dict[f"test_{test_length}_days"] = mape_met(eval_subset['y'].values, eval_subset['forecast'].values)
                results.append({
                    "Model": model_name,
                    "Transforms": stringify_transform(list(transform_combination)),
                    **error_dict
                })
                print(f"{model_name} MAPE: {error_dict[f'test_{max_test_length}_days']:.2f}% with transforms {stringify_transform(list(transform_combination))}")
            except Exception as e:
                print(f"Skipping combination {fit_num + 1} due to error: {e}")
            fit_num += 1

    return pd.DataFrame(results)


In [ ]:
from joblib import Parallel, delayed
import multiprocessing
import time

def process_scenario(sensor_name, scenario_name, scenario, selected_sensors_df):
    """ Process each scenario independently and save results. """
    print(f'{sensor_name}_{scenario_name}')
    formatted_df = format_df_to_mlforecast(selected_sensors_df[['full_date', sensor_name]], 'full_date', sensor_name, unique_id=sensor_name)
    formatted_df = formatted_df[['ds', 'y', 'unique_id']]
    
    train_df, test_df = split_data(formatted_df, scenario)
    target_transforms = get_dynamic_transforms(train_df)

    results = evaluate_models_statsforecast(train_df, test_df, target_transforms)

    save_results(results, f"results/run_5/{sensor_name}_{scenario_name}.csv")

    return results

def run_all_scenarios_parallel(scenarios_sensors, selected_sensors_df):
    # don't use all cpus (instead all but one)
    n_jobs = max(1, multiprocessing.cpu_count() - 1)
    results = Parallel(n_jobs=n_jobs)( 
        delayed(process_scenario)(sensor_name, scenario_name, scenario, selected_sensors_df )
        for sensor_name, scenarios in scenarios_sensors.items()
        for scenario_name, scenario in scenarios.items()
    )

    return results


In [18]:
results = run_all_scenarios_parallel(scenarios_sensors, selected_sensors_df)